In [28]:
import numpy as np
import pandas as pd
from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [72]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr

In [14]:
def post_processing(pred):
    new_pred=[]
    for i in pred:
        i=i.lower()
        if 'response' in i:
            try: new_pred.append(i.split('response')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        elif 'output' in i:
            try: new_pred.append(i.split('output')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        else:
            try: new_pred.append(i.split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [45]:
ds = load_dataset("beanham/spatial_join_dataset")
test=ds['test']
gt=np.array(test['label'])

In [48]:
models = ['llama3', '4o_mini', 'gemini', 'qwen']
methods = ['zero_shot', 'few_shot'] 
modes = ['no_heur', 'with_heur_hint', 'with_heur_value']
heuristics = ['angle', 'distance', 'comb']
configs=['_'.join(i) for i in list(product(methods, modes, heuristics))]
configs.remove('zero_shot_no_heur_angle')
configs.remove('zero_shot_no_heur_distance')
configs.remove('zero_shot_no_heur_comb')
configs.remove('few_shot_no_heur_angle')
configs.remove('few_shot_no_heur_distance')
configs.remove('few_shot_no_heur_comb')
configs.insert(0, 'zero_shot_no_heur')
configs.insert(7, 'few_shot_no_heur')

In [73]:
results=[]
for model in models:
    print(f'Model: {model}...')
    for config in configs:
        pred=np.load(f'base/{model}/{model}_{config}.npy')
        pred=post_processing(pred)
        metrics=metric_calculation(pred, gt)
        results.append([config, model, metrics[0], metrics[1]])
results=pd.DataFrame(results, columns=['config', 'model', 'acc', 'f1'])

Model: llama3...
Model: 4o_mini...
Model: gemini...
Model: qwen...


In [74]:
results[results.model=='qwen']

,config,model,acc,f1
42,zero_shot_no_heur,qwen,0.603128,0.391103
43,zero_shot_with_heur_hint_angle,qwen,0.639622,0.485164
44,zero_shot_with_heur_hint_distance,qwen,0.605409,0.389906
45,zero_shot_with_heur_hint_comb,qwen,0.659824,0.535232
46,zero_shot_with_heur_value_angle,qwen,0.940371,0.936432
47,zero_shot_with_heur_value_distance,qwen,0.754969,0.694764
48,zero_shot_with_heur_value_comb,qwen,0.948843,0.945793
49,few_shot_no_heur,qwen,0.632454,0.563067
50,few_shot_with_heur_hint_angle,qwen,0.685891,0.660780
51,few_shot_with_heur_hint_distance,qwen,0.680678,0.657336
